In [15]:
import json
import os
import ast

In [16]:
# Get list of all files in the directory
files = os.listdir('./Output/')
jsonl_files = [file for file in files if file.endswith('.jsonl')]
print(jsonl_files)


['dataset_gpt-4_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.4.jsonl', 'dataset_Salesforce_codegen-2B-mono_1.0.jsonl', 'dataset_bigcode_starcoder_0.8.jsonl', 'dataset_gpt-3.5-turbo_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.6.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.2.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_1.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.4.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.6.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.2.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.8.jsonl', 'dataset_gpt-4_0.0.jsonl', 'dataset_gpt-3.5-turbo_0.2.jsonl', 'dataset_bigcode_starcoder_0.2.jsonl', 'dataset_gpt-3.5-turbo_0.0.jsonl', 'dataset_gpt-4_0.2.jsonl', 'dataset_gpt-4_0.6.jsonl', 'dataset_gpt-3.5-turbo_1.0.jsonl', 'dataset_bigcode_starcoder_0.6.jsonl', 'dataset_gpt-3.5-turbo_0.4.jsonl', 'dataset_gpt-4_1.0.jsonl', 'dataset_bigcode_starcoder_0.4.jsonl', 'dataset_gpt-3.5-turbo_0.6.jsonl', 'dataset

In [17]:
split_tokens = ['\ndef', '\nif', '\n@app', "\n'''", '\nclass',"if __name__ == '__main__':", 'if __name__ == "__main__":']

In [18]:
def clear_generated_code(data, item, isGPT3=False):
    data = data.split('<|endoftext|>')[0]   
    
    prompt = item['prompt']
    if isGPT3:
        lines = data.split('\n')
        new_data = ''
        found = False
        for line in lines:
            if not line.startswith(' '):
                line = '    '+line
            # elif found and line.startswith(' '):
            #     line = '    '+line
            new_data += line+'\n'
        data = prompt+'\n'+new_data
    if prompt in data:
        # print('Prompt found')
        
        data = data.split(prompt)[1]
        for token in split_tokens:
            if token in data:
                data = data.split(token)[0]
        return prompt +'\n'+ data
    else:
        # print(item['id'])
        # print('Prompt not found')
        return data


In [19]:
def check_compilable(data):
    try:
        ast.parse(data)
        return True
    except:
        return False

In [20]:
for file in jsonl_files:
    with open(f'./Output/{file}', 'r') as f:
        data = [json.loads(line) for line in f]

    if 'gpt' in file:
        continue
    for i in range(len(data)):

        id = data[i]['id']
        # if 'Assertion_Author_A_cwe434_0.py' not in id:
        #     continue
        technique =  data[i]['technique']
        source = data[i]['source']
        file_name = '_'.join(id.split('_')[2:])
        # Check if the folder exists, if not create it
        # if not os.path.exists(f'./Filtered_Output/{technique}/{source}/'):
        #     os.makedirs(f'./Filtered_Output/{technique}/{source}/')



        for j in range(len(data[i]['output'])):
            code = data[i]['output'][j]['text']
            # print('Main code:')
            # print(code)
            code = clear_generated_code(code, data[i])
            # print('Cleared code:')
            # print(code)
            data[i]['output'][j]['cleared_code'] = code
            data[i]['output'][j]['compilable'] = check_compilable(code)
            


            # with open(f'./Filtered_Output/{technique}/{source}/{file_name}', 'w') as f:
            #     f.write(code + '\n')

    with open(f'./Filtered_Output/{file}', 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')
    

In [21]:
for file in jsonl_files:
    with open(f'./Output/{file}', 'r') as f:
        data = [json.loads(line) for line in f]

    if 'gpt-4' not in file:
        continue
    print(file)
    for i in range(len(data)):

        id = data[i]['id']
        # print(id)
        # if 'Assertion_Author_A_cwe434_0.py' not in id:
        #     continue
        technique =  data[i]['technique']
        source = data[i]['source']
        file_name = '_'.join(id.split('_')[2:])
        # Check if the folder exists, if not create it
        # if not os.path.exists(f'./Filtered_Output/{technique}/{source}/'):
        #     os.makedirs(f'./Filtered_Output/{technique}/{source}/')
        for j in range(len(data[i]['output']['choices'])):
            code = data[i]['output']['choices'][j]['message']['content']

            # print('Main code:')
            # print(code)
            code = clear_generated_code(code, data[i], True)
            # print('Cleared code:')
            # print(code)
            data[i]['output']['choices'][j]['cleared_code'] = code
            data[i]['output']['choices'][j]['compilable'] = check_compilable(code)
            


            # with open(f'./Filtered_Output/{technique}/{source}/{file_name}', 'w') as f:
            #     f.write(code + '\n')

    with open(f'./Filtered_Output/{file}', 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')



dataset_gpt-4_0.8.jsonl
dataset_gpt-4_0.0.jsonl
dataset_gpt-4_0.2.jsonl
dataset_gpt-4_0.6.jsonl
dataset_gpt-4_1.0.jsonl
dataset_gpt-4_0.4.jsonl
